# Class 2. Quantum Gates, Circuits and Entanglement

EVA: Quantum Machine Learning | ZHAW | Pavel Sulimov

---

Goals of this practice session:

1. Apply single-qubit gates (Pauli, Hadamard, rotations) via matrix multiplication.
2. Work with the tensor product and multi-qubit systems.
3. Build entangled states (Bell, GHZ) and verify them through measurement.
4. Trace state evolution through a circuit step by step.

**Convention reminder.** Unless stated otherwise, every qubit in a quantum circuit starts in the state $|0\rangle = \begin{pmatrix} 1 \\ 0 \end{pmatrix}$. When we write "apply gate $G$", we mean $G|0\rangle$ unless a different initial state is specified. To start from a different state, apply preparation gates first (e.g., $X$ for $|1\rangle$, $H$ for $|+\rangle$).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

I2 = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
S = np.array([[1, 0], [0, 1j]])
T = np.array([[1, 0], [0, np.exp(1j * np.pi / 4)]])

ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])

for name, gate in [("X", X), ("H", H), ("Z", Z)]:
    print(f"{name} =\n{np.round(gate, 4)}\n")

---
## Part 1: Math tasks

### M2.1. Gate action on basis states

Compute $X|0\rangle$, $X|1\rangle$, $H|0\rangle$, $H|1\rangle$ by matrix multiplication.

In [ ]:
print("Pauli X (bit-flip)")
print(f"  X|0> = {X @ ket_0}   i.e. |1>")
print(f"  X|1> = {X @ ket_1}   i.e. |0>")

print("\nHadamard")
print(f"  H|0> = {np.round(H @ ket_0, 4)}   i.e. |+>")
print(f"  H|1> = {np.round(H @ ket_1, 4)}   i.e. |->")

print("\nPauli Y")
print(f"  Y|0> = {Y @ ket_0}   i.e. i|1>")
print(f"  Y|1> = {Y @ ket_1}   i.e. -i|0>")

print("\nPauli Z (phase-flip)")
print(f"  Z|0> = {Z @ ket_0}   unchanged")
print(f"  Z|1> = {Z @ ket_1}   i.e. -|1>")

### M2.2. Gate identity: HXH = Z

Show that $HXH = Z$ by matrix multiplication. Conjugation by $H$ maps between the $X$- and $Z$-eigenbases.

In [ ]:
result = H @ X @ H
print("H X H =")
print(np.round(result, 10))
print(f"\nEquals Z? {np.allclose(result, Z)}")

# Related identities
print(f"H Z H equals X? {np.allclose(H @ Z @ H, X)}")
print(f"H Y H equals -Y? {np.allclose(H @ Y @ H, -Y)}")

### M2.3. Creating a Bell state step by step

Compute $\text{CNOT}\,(H \otimes I)|00\rangle$ step by step and verify that the result is $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$.

Steps: (1) write $|00\rangle$ as a 4-vector, (2) apply $H \otimes I$, (3) apply CNOT.

In [ ]:
ket_00 = np.kron(ket_0, ket_0)
print(f"Step 1:  |00> = {ket_00}")

H_kron_I = np.kron(H, I2)
state_after_H = H_kron_I @ ket_00
print(f"Step 2:  (H x I)|00> = {np.round(state_after_H, 4)}")

# CNOT: |00>->|00>, |01>->|01>, |10>->|11>, |11>->|10>
CNOT = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
])

bell_state = CNOT @ state_after_H
print(f"Step 3:  CNOT . (H x I)|00> = {np.round(bell_state, 4)}")
print(f"         = (1/sqrt2)(|00> + |11>)  i.e. |Phi+>")

### M2.4. Proving entanglement

Show that $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ cannot be written as a tensor product of two single-qubit states.

Proof strategy: assume $|\Phi^+\rangle = (a|0\rangle + b|1\rangle) \otimes (c|0\rangle + d|1\rangle)$ and derive a contradiction.

In [ ]:
# Proof by contradiction:
# Assume |Phi+> = (a|0> + b|1>) x (c|0> + d|1>)
#   => ac = 1/sqrt2,  ad = 0,  bc = 0,  bd = 1/sqrt2
# From ad = 0: a = 0 or d = 0.
#   a = 0 => ac = 0, contradicts ac = 1/sqrt2.
#   d = 0 => bd = 0, contradicts bd = 1/sqrt2.
# Therefore |Phi+> is not separable.

print("Proof by contradiction:")
print("Assume |Phi+> = (a|0>+b|1>) x (c|0>+d|1>)")
print("=> ac=1/sqrt2, ad=0, bc=0, bd=1/sqrt2")
print("ad=0 => a=0 or d=0")
print("  a=0 => ac=0, contradiction.")
print("  d=0 => bd=0, contradiction.")
print("=> |Phi+> is entangled.")

# Numerical check via Schmidt decomposition (SVD of reshaped state)
bell_matrix = bell_state.reshape(2, 2)
_, s, _ = np.linalg.svd(bell_matrix)
schmidt_rank = int(np.sum(s > 1e-10))
print(f"\nSingular values: {np.round(s, 4)}")
print(f"Schmidt rank: {schmidt_rank}  (rank > 1 => entangled)")

---
## Part 2: Programming tasks

### P2.1. All four Bell states (Qiskit)

Build 2-qubit circuits creating all four Bell states and run them with `StatevectorSampler`.

| State | Formula | Circuit |
|---|---|---|
| $\lvert\Phi^+\rangle$ | $(\lvert00\rangle + \lvert11\rangle)/\sqrt{2}$ | H(0), CX(0,1) |
| $\lvert\Phi^-\rangle$ | $(\lvert00\rangle - \lvert11\rangle)/\sqrt{2}$ | H(0), Z(0), CX(0,1) |
| $\lvert\Psi^+\rangle$ | $(\lvert01\rangle + \lvert10\rangle)/\sqrt{2}$ | H(0), CX(0,1), X(1) |
| $\lvert\Psi^-\rangle$ | $(\lvert01\rangle - \lvert10\rangle)/\sqrt{2}$ | H(0), Z(0), CX(0,1), X(1) |

In [ ]:
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from qiskit.quantum_info import Statevector


def make_bell_circuits() -> dict[str, QuantumCircuit]:
    """Return circuits for all four Bell states."""
    specs = {
        "Phi+": [("h", 0), ("cx", (0, 1))],
        "Phi-": [("h", 0), ("z", 0), ("cx", (0, 1))],
        "Psi+": [("h", 0), ("cx", (0, 1)), ("x", 1)],
        "Psi-": [("h", 0), ("z", 0), ("cx", (0, 1)), ("x", 1)],
    }
    circuits = {}
    for name, gates in specs.items():
        qc = QuantumCircuit(2, name=name)
        for gate, args in gates:
            if isinstance(args, tuple):
                getattr(qc, gate)(*args)
            else:
                getattr(qc, gate)(args)
        circuits[name] = qc
    return circuits


bell_circuits = make_bell_circuits()
basis = ["00", "01", "10", "11"]

for name, qc in bell_circuits.items():
    sv = Statevector.from_instruction(qc)
    probs = dict(zip(basis, np.round(sv.probabilities(), 4)))
    print(f"|{name}>:  sv = {np.round(sv.data, 4)}  P = {probs}")
    print(qc.draw("text"), "\n")

In [ ]:
sampler = StatevectorSampler()

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for idx, (name, qc) in enumerate(bell_circuits.items()):
    qc_meas = qc.copy()
    qc_meas.measure_all()

    counts = sampler.run([qc_meas], shots=1024).result()[0].data.meas.get_counts()

    ax = axes.flat[idx]
    labels = sorted(counts.keys())
    values = [counts.get(l, 0) for l in labels]
    colors = ["steelblue", "coral", "seagreen", "orange"]
    bars = ax.bar(labels, values, color=colors[: len(labels)])
    ax.set_title(f"|{name}>", fontsize=14)
    ax.set_ylabel("counts")
    ax.set_ylim(0, 1100)
    for bar, val in zip(bars, values):
        ax.text(
            bar.get_x() + bar.get_width() / 2.0,
            bar.get_height() + 20,
            str(val), ha="center",
        )

plt.suptitle("Bell states (1024 shots)", fontsize=15)
plt.tight_layout()
plt.show()

### P2.2. Parameterized rotation and expectation values (PennyLane)

Implement $R_y(\theta)|0\rangle$ and plot $\langle Z \rangle$ as a function of $\theta \in [0, 2\pi]$. Analytically, $\langle Z \rangle = \cos\theta$.

In [ ]:
import pennylane as qml

dev = qml.device("default.qubit", wires=1)


@qml.qnode(dev)
def ry_expval(theta):
    """Return <Z> after Ry(theta) on |0>."""
    qml.RY(theta, wires=0)
    return qml.expval(qml.PauliZ(0))


thetas = np.linspace(0, 2 * np.pi, 100)
expvals = np.array([float(ry_expval(t)) for t in thetas])

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(thetas, expvals, "b-", lw=2.5, label=r"$\langle Z\rangle$ (circuit)")
ax.plot(thetas, np.cos(thetas), "r--", lw=2, label=r"$\cos\theta$ (analytical)")
ax.set_xlabel(r"$\theta$ (rad)")
ax.set_ylabel(r"$\langle Z\rangle$")
ax.set_title(r"$\langle Z\rangle$ for $R_y(\theta)|0\rangle$")
ax.legend()
ax.set_xticks([0, np.pi / 2, np.pi, 3 * np.pi / 2, 2 * np.pi])
ax.set_xticklabels(["0", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"])
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

max_err = np.max(np.abs(expvals - np.cos(thetas)))
print(f"Max |error| vs cos(theta): {max_err:.2e}")

### P2.3. GHZ state (Qiskit)

Build the 3-qubit GHZ state $|GHZ\rangle = \frac{1}{\sqrt{2}}(|000\rangle + |111\rangle)$ and verify it through measurement. The GHZ state generalises Bell states to $n$ qubits.

In [ ]:
qc_ghz = QuantumCircuit(3)
qc_ghz.h(0)
qc_ghz.cx(0, 1)
qc_ghz.cx(0, 2)
print(qc_ghz.draw("text"))

sv_ghz = Statevector.from_instruction(qc_ghz)
print(f"\nStatevector: {np.round(sv_ghz.data, 4)}")

qc_ghz_meas = qc_ghz.copy()
qc_ghz_meas.measure_all()
counts = sampler.run([qc_ghz_meas], shots=2048).result()[0].data.meas.get_counts()

fig, ax = plt.subplots(figsize=(8, 4))
labels = sorted(counts.keys())
values = [counts[l] for l in labels]
ax.bar(labels, values, color=["steelblue", "coral"])
ax.set_title("3-qubit GHZ state (2048 shots)", fontsize=14)
ax.set_ylabel("counts")
for bar, val in zip(ax.patches, values):
    ax.text(
        bar.get_x() + bar.get_width() / 2.0,
        bar.get_height() + 30,
        str(val), ha="center",
    )
plt.tight_layout()
plt.show()

### P2.4. State evolution through a circuit

Trace the probability distribution after each gate in the Bell-state circuit.

In [ ]:
basis_labels = ["00", "01", "10", "11"]

state = Statevector.from_label("00")
snapshots = [("initial |00>", state.probabilities().copy())]

qc_h = QuantumCircuit(2)
qc_h.h(0)
state = state.evolve(qc_h)
snapshots.append(("after H(q0)", state.probabilities().copy()))

qc_cx = QuantumCircuit(2)
qc_cx.cx(0, 1)
state = state.evolve(qc_cx)
snapshots.append(("after CX(0,1)", state.probabilities().copy()))

for label, probs in snapshots:
    p = dict(zip(basis_labels, np.round(probs, 4)))
    print(f"{label:18s}  P = {p}")

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for idx, (title, probs) in enumerate(snapshots):
    ax = axes[idx]
    vals = [probs[i] for i in range(4)]
    colors = ["steelblue" if v > 0 else "lightgray" for v in vals]
    ax.bar(basis_labels, vals, color=colors)
    ax.set_title(title, fontsize=13)
    ax.set_ylabel("probability")
    ax.set_ylim(0, 1.1)

plt.suptitle("State evolution: |00> -> H -> CX -> Bell state", fontsize=14)
plt.tight_layout()
plt.show()

---
## Bonus

### Bonus 1. Rotation gate matrices

$R_x(\theta)$, $R_y(\theta)$, $R_z(\theta)$ are parameterized unitaries. In QML the angles $\theta$ play the role of trainable weights.

In [ ]:
def Rx(theta):
    c, s = np.cos(theta / 2), np.sin(theta / 2)
    return np.array([[c, -1j * s], [-1j * s, c]])


def Ry(theta):
    c, s = np.cos(theta / 2), np.sin(theta / 2)
    return np.array([[c, -s], [s, c]])


def Rz(theta):
    return np.diag([np.exp(-1j * theta / 2), np.exp(1j * theta / 2)])


# At theta = pi these reduce to -i times the Pauli gate (up to global phase)
print("Rx(pi) equals -iX?", np.allclose(Rx(np.pi), -1j * X))
print("Ry(pi) equals -iY?", np.allclose(Ry(np.pi), -1j * Y))
print("Rz(pi) equals -iZ?", np.allclose(Rz(np.pi), -1j * Z))

# Ry(pi/2)|0> vs H|0>: same probabilities, different phases
psi_ry = Ry(np.pi / 2) @ ket_0
psi_h = H @ ket_0
print(f"\nRy(pi/2)|0> = {np.round(psi_ry, 4)}")
print(f"H|0>        = {np.round(psi_h, 4)}")

### Bonus 2. Parameterized circuit as a function

A parameterized 2-qubit circuit maps rotation angles to output probabilities, analogously to a neural-network layer mapping weights to activations.

In [ ]:
dev2 = qml.device("default.qubit", wires=2)


@qml.qnode(dev2)
def quantum_layer(params):
    """Two layers of Ry rotations with a CNOT in between."""
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(params[2], wires=0)
    qml.RY(params[3], wires=1)
    return qml.probs(wires=[0, 1])


param_sweep = np.linspace(0, 2 * np.pi, 50)
fixed = [np.pi / 4, np.pi / 3, np.pi / 6]

results = np.array([quantum_layer([p] + fixed) for p in param_sweep])

fig, ax = plt.subplots(figsize=(10, 5))
labels_out = ["|00>", "|01>", "|10>", "|11>"]
colors = ["steelblue", "coral", "seagreen", "orange"]
for j in range(4):
    ax.plot(param_sweep, results[:, j], lw=2, color=colors[j],
            label=labels_out[j])

ax.set_xlabel(r"$\theta_0$")
ax.set_ylabel("probability")
ax.set_title("Output probabilities vs first rotation angle")
ax.legend()
ax.set_xticks([0, np.pi / 2, np.pi, 3 * np.pi / 2, 2 * np.pi])
ax.set_xticklabels(["0", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"])
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## Summary

Quantum gates are unitary matrices acting on qubit states. The Pauli gates $X, Y, Z$ and Hadamard $H$ are fixed, while the rotation gates $R_x, R_y, R_z$ carry a continuous parameter $\theta$, the analogue of a trainable weight in classical ML.

Multi-qubit state spaces grow as $2^n$ via the tensor product. Entanglement (e.g. Bell and GHZ states) produces correlations that have no classical counterpart. Circuits can be viewed as computational graphs analogous to neural-network layers.

Next session: measurement, observables, and expectation values.